In [19]:
import numpy as np
import pandas as pd
from sphinx.cmd.quickstart import suffix

from util.data_loader import DataLoader
movielens = DataLoader().main()
movie_stats = movielens.train.groupby(["movie_id","title"]).agg({"rating":[np.size, np.mean]})
movie_stats=movie_stats.sort_values(by=("rating","mean"),ascending=False)
movie_stats.head()


C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

rating     
                                     size mean
movie_id title                                
4095     Cry Freedom (1987)             1  5.0
4043     At Close Range (1986)          1  5.0
32153    Once Upon a Forest (1993)      1  5.0
31705    Beautiful Boxer (2003)         1  5.0
868      Death in Brunswick (1991)      1  5.0

In [20]:
movie_rates = movielens.train.groupby(['movie_id','title']).agg({"rating":[np.size,np.mean]})
least_size = movie_rates['rating']['size']>=100
least_size_movie_rate = movie_rates[least_size].sort_values(by=('rating','mean'), ascending=False)
least_size_movie_rate.head()


C:\Users\USER\AppData\Local\Temp\ipykernel_3440\133856673.py:1: FutureWarning: The provided callable <function mean at 0x00000235C78DC7C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_rates = movielens.train.groupby(['movie_id','title']).agg({"rating":[np.size,np.mean]})


rating          
                                                       size      mean
movie_id title                                                       
318      Shawshank Redemption, The (1994)               423  4.492908
50       Usual Suspects, The (1995)                     332  4.459337
912      Casablanca (1942)                              163  4.444785
904      Rear Window (1954)                             129  4.441860
2019     Seven Samurai (Shichinin no samurai) (1954)    104  4.408654

In [25]:
from util.models import RecommendResult, Dataset
from base_recommender import BaseRecommender
from collections import defaultdict
import numpy as np
np.random.seed(0)

class PopularityRecommender(BaseRecommender):
    def recommend(self, dataset, **kwargs):
        rating_size = kwargs.get("rating_size",200)
        train_rate_avg = dataset.train.groupby("movie_id").agg({"rating":np.mean})
        pred_rate = dataset.test.merge(train_rate_avg,on="movie_id", how="left", suffixes = ("_train","_pred")).fillna(0)

        #유저가 본영화 -> movie_id 별로 평균과 크기 movie-stats,-> valid_movies들 ->valid중 평가 높은순으로 movie_id 리스트로
        #유저 돌아가면서 movie_id 평가 높은순으로 정렬도닌거에서 하나씩 뽑으ㅕㄴ서 유저가 안본거면 좋아할거라고 예측하기
        pred_love_items = defaultdict(list)
        user_watched_movies = dataset.train.groupby("user_id").agg({"movie_id":list})["movie_id"].to_dict()
        movie_stats = dataset.train.groupby("movie_id").agg({"rating":[np.mean,np.size]})
        valid_movies = movie_stats['rating']['size']>=rating_size
        sorted_valid_movies = movie_stats[valid_movies].sort_values(by=("rating","mean"), ascending=False).index.tolist()
        for user_id in dataset.train.user_id.unique():
            for movie_id in sorted_valid_movies:
                if movie_id not in user_watched_movies[user_id]:
                    pred_love_items[user_id].append(movie_id)
                if len(pred_love_items[user_id])==10:
                    break
        return RecommendResult(pred_rate.rating_pred, pred_love_items)

if __name__ =="__main__":
    PopularityRecommender().eval()

C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

rmse: 1.089, recall: 0.040, precision:0.013


C:\Users\USER\AppData\Local\Temp\ipykernel_3440\676802219.py:10: FutureWarning: The provided callable <function mean at 0x00000235C78DC7C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  train_rate_avg = dataset.train.groupby("movie_id").agg({"rating":np.mean})
C:\Users\USER\AppData\Local\Temp\ipykernel_3440\676802219.py:31: FutureWarning: The provided callable <function mean at 0x00000235C78DC7C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating":[np.mean,np.size]})
